<a href="https://colab.research.google.com/github/swethaukkarde/Ai-powered-resume-job-matching/blob/main/hackcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lightgbm

In [1]:
!pip -q install lightgbm pandas numpy scikit-learn

import pandas as pd
import lightgbm as lgb
num_candidates = int(input("Enter number of candidates for this job: "))
candidates = []
match_scores = []
verified_skills = []
ats_scores = []
for i in range(num_candidates):
    print(f"\nCandidate {i+1}")
    candidates.append(input("Candidate name: "))
    match_scores.append(float(input("Match score (0–1): ")))
    verified_skills.append(float(input("Verified skills % (0–1): ")))
    ats_scores.append(float(input("ATS score (0–100): ")))
df = pd.DataFrame({
    "candidate": candidates,
    "job_id": ["JOB1"] * num_candidates,
    "match_score": match_scores,
    "verified_skills": verified_skills,
    "ATS_score": ats_scores
})
df["baseline_score"] = (
    0.4 * df["match_score"] +
    0.4 * df["verified_skills"] +
    0.2 * (df["ATS_score"] / 100)
)
df["label"] = df["baseline_score"].rank(method="dense").astype(int)
X = df[["match_score", "verified_skills", "ATS_score"]]
y = df["label"]
group = [len(df)]
train_data = lgb.Dataset(X, label=y, group=group)
params = {
    "objective": "lambdarank",
    "metric": "ndcg",
    "learning_rate": 0.05,
    "num_leaves": 31,
    "verbosity": -1
}
model = lgb.train(params, train_data, num_boost_round=50)

df["lgbm_score"] = model.predict(X)

df["final_rank"] = (
    df["lgbm_score"]
    .rank(ascending=False, method="dense")
)

df = df.sort_values("final_rank")
print("\n===== FINAL RANKING =====\n")
print(df[[
    "candidate",
    "match_score",
    "verified_skills",
    "ATS_score",
    "final_rank"
]])

Enter number of candidates for this job: 1

Candidate 1
Candidate name: X
Match score (0–1): 1
Verified skills % (0–1): 1
ATS score (0–100): 42

===== FINAL RANKING =====

  candidate  match_score  verified_skills  ATS_score  final_rank
0         X          1.0              1.0       42.0         1.0


In [2]:
!pip -q install lightgbm pandas numpy scikit-learn
import pandas as pd
import lightgbm as lgb
num_candidates = int(input("Enter number of candidates for this job: "))
candidates = []
match_scores = []
verified_skills = []
ats_scores = []
for i in range(num_candidates):
    print(f"\nCandidate {i+1}")
    candidates.append(input("Candidate name: "))
    match_scores.append(float(input("Match score (0–1): ")))
    verified_skills.append(float(input("Verified skills % (0–1): ")))
    ats_scores.append(float(input("ATS score (0–100): ")))
df = pd.DataFrame({
    "candidate": candidates,
    "job_id": ["JOB1"] * num_candidates,
    "match_score": match_scores,
    "verified_skills": verified_skills,
    "ATS_score": ats_scores
})
df["baseline_score"] = (
    0.4 * df["match_score"] +
    0.4 * df["verified_skills"] +
    0.2 * (df["ATS_score"] / 100)
)
df["label"] = pd.qcut(
    df["baseline_score"],
    q=min(3, len(df)),
    labels=range(1, min(3, len(df)) + 1)
).astype(int)
X = df[["match_score", "verified_skills", "ATS_score"]]
y = df["label"]
group = [len(df)]
train_data = lgb.Dataset(X, label=y, group=group)
params = {
    "objective": "lambdarank",
    "metric": "ndcg",
    "learning_rate": 0.05,
    "num_leaves": 31,
    "verbosity": -1
}

model = lgb.train(params, train_data, num_boost_round=50)
df["lgbm_score"] = model.predict(X)
df["rank"] = df["lgbm_score"].rank(
    ascending=False,
    method="first"
)
max_rank = df["rank"].max()
df["rank"] = (max_rank + 1) - df["rank"]
df = df.sort_values("rank", ascending=False)

print("\n===== FINAL RANKING =====\n")
print(df[[
    "candidate",
    "match_score",
    "verified_skills",
    "ATS_score",
    "rank"
]])


Enter number of candidates for this job: 2

Candidate 1
Candidate name: x
Match score (0–1): 1
Verified skills % (0–1): 1
ATS score (0–100): 42

Candidate 2
Candidate name: y
Match score (0–1): 1
Verified skills % (0–1): 0
ATS score (0–100): 87

===== FINAL RANKING =====

  candidate  match_score  verified_skills  ATS_score  rank
0         x          1.0              1.0       42.0   2.0
1         y          1.0              0.0       87.0   1.0


In [3]:
!pip -q install transformers torch pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 80.1 MB/s eta 0:00:00


In [5]:
import pdfplumber
import torch
from transformers import AutoTokenizer, AutoModel
from google.colab import files
print("Upload RESUME PDF")
resume_file = files.upload()
print("\nUpload JOB DESCRIPTION PDF")
jd_file = files.upload()
resume_path = list(resume_file.keys())[0]
jd_path = list(jd_file.keys())[0]
def extract_text(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() or ""
    return text.strip()
resume_text = extract_text(resume_path)
jd_text = extract_text(jd_path)
print("\n--- Resume Text Extracted ---")
print(resume_text[:500], "...")
print("\n--- Job Description Text Extracted ---")
print(jd_text[:500], "...")
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModel.from_pretrained("bert-base-multilingual-cased")
def get_embedding(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=512
    )
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1)
    return embedding
resume_embedding = get_embedding(resume_text)
jd_embedding = get_embedding(jd_text)
print("\n===== RESUME EMBEDDING =====")
print(resume_embedding)
print("Shape:", resume_embedding.shape)
print("\n===== JOB DESCRIPTION EMBEDDING =====")
print(jd_embedding)
print("Shape:", jd_embedding.shape)

Upload RESUME PDF


Saving Kannada_Resume.pdf to Kannada_Resume.pdf

Upload JOB DESCRIPTION PDF


Saving Kannada_Resume.pdf to Kannada_Resume (1).pdf

--- Resume Text Extracted ---
nnnnnnnnnnn (Resume)
nnnnn:
nnnnn:
nnnnnnn nnnnnn:
nnnnn:
nnnnnn
nnnnn nnnnnnnnnnn nnnn nnnnnn nnnnn nnnnnnnnnn nnnnnnnnn
nnnnnn nnnnnnnn nnnnnnnnnn.
nnnnnnnn nnnnnn
(cid:127) nnnn –
(cid:127) nnnnnn/nnnnnnnnnnnnn –
(cid:127) nnnn –
nnnnnnnn nnnnnnnnn
(cid:127)
(cid:127)
nnnnn (nnnnnn)
(cid:127)
nnnnnnnn nnnnnnn
nnnn nnnnnn:
nnnn:
nnnnnnn: nnnnn, nnnnnnnn
nnnnn
nnnnn nnnnn nnnnnnnnn nnnn nnnnnnnnnnnnn nnnnnnnnnn.
nnnnnn:
nnnn:
nnnnnnnnn: ...

--- Job Description Text Extracted ---
nnnnnnnnnnn (Resume)
nnnnn:
nnnnn:
nnnnnnn nnnnnn:
nnnnn:
nnnnnn
nnnnn nnnnnnnnnnn nnnn nnnnnn nnnnn nnnnnnnnnn nnnnnnnnn
nnnnnn nnnnnnnn nnnnnnnnnn.
nnnnnnnn nnnnnn
(cid:127) nnnn –
(cid:127) nnnnnn/nnnnnnnnnnnnn –
(cid:127) nnnn –
nnnnnnnn nnnnnnnnn
(cid:127)
(cid:127)
nnnnn (nnnnnn)
(cid:127)
nnnnnnnn nnnnnnn
nnnn nnnnnn:
nnnn:
nnnnnnn: nnnnn, nnnnnnnn
nnnnn
nnnnn nnnnn nnnnnnnnn nnnn nnnnnnnnnnnnn nnnnnnnnnn.
nnnnnn:
nnnn:


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: bert-base-multilingual-cased
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



===== RESUME EMBEDDING =====
tensor([[-5.3643e-01, -3.2799e-01, -2.8225e-01,  7.9829e-01, -2.6674e-01,
          2.1723e-01, -8.1383e-01, -1.0817e-01,  3.7598e-01,  6.5104e-01,
          6.4338e-01,  6.5480e-01,  8.0165e-01,  1.5202e-01, -3.5834e-01,
         -2.9679e-02,  3.0464e-01, -4.9636e-01,  2.4532e-01,  5.2285e-01,
         -9.5785e-02,  5.0042e-01,  4.3751e-01,  2.7250e-01,  9.2193e-01,
          3.5348e-01, -4.1957e-01,  1.1158e-02, -1.3091e-01,  1.1064e-01,
         -4.6366e-01,  5.3370e-01, -8.7197e-02,  1.6315e+00, -1.5571e-01,
         -4.3226e-02, -1.5865e-01,  8.0749e-02, -9.3600e-02, -8.5010e-01,
          1.9650e-01,  7.1360e-01,  1.0746e-01,  2.6674e-01,  4.2435e-02,
         -3.6401e-01,  3.8019e-01,  3.5723e-01, -2.1762e-01,  5.9204e-01,
          3.9594e-01, -6.0083e-02,  2.5680e-01,  4.8202e-01, -8.3389e-01,
         -7.7477e-01, -3.8469e-01,  1.7952e-01,  3.6077e-01, -3.6390e-02,
          2.4929e-01, -1.2292e-01, -1.5398e-01, -5.3199e-01,  6.4253e-02,
        